# This notebook creates the flight plans for a set of OD pairs stored in a csv file
The .csv file must contain the following fields for each drone:
 - Origin
 - Destination
 - Departure time
 - Type of drone
 
The notebook will provide scenario files for each drone.

TODO: include strategic deconfliction

In [1]:
"""
We define a default path where the modules developed in USEPE are stored
"""

default_path = r"C:\workspace3\bluesky"

"""
We define a second default path where the scenarios are stored
"""

default_path2 = r"C:\workspace3\scenarios-exercise1\scenario\USEPE\exercise_1"


In [2]:
import os
import sys
import configparser
import pickle

os.chdir(default_path)
from usepe.city_model.multi_di_graph_3D import MultiDiGrpah3D
from usepe.city_model.utils import read_my_graphml
from usepe.city_model.city_graph import cityGraph
from usepe.city_model.scenario_definition import createFlightPlansFromCSV

c:\workspace3\python3_env\lib\site-packages\geopandas\_compat.py:106: UserWarning: The Shapely GEOS version (3.8.1-CAPI-1.13.3) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


Configuration file¶

We create a config file with the following fields:

[City]

mode = rectangle

#hannover_lat = 52.376

#hannover_lon = 9.76

#zone_size = 1000

hannover_lat_min = 52.297839999994714

hannover_lat_max = 52.438260000005286

hannover_lon_min = 9.643080000001035

hannover_lon_max = 9.844349999998968

import = True

imported_graph_path = ./data/exercise_1_reference.graphml

[Layers]
number_of_layers = 9

layer_width = 25

[BuildingData]

lat_min = 52.297839999994714

lat_max = 52.438260000005286

lon_min = 9.643080000001035

lon_max = 9.844349999998968

divisions = 20

directory_hannover = C:\workspace3\bluesky_usepe_git\usepe\city_model\data\CityGML_LoD1

[Options]

one_way = False

simplify = True

simplification_distance = 0

simplification_error = 2

[Outputs]

graph_path = ./data/exercise_1_reference.graphml

[Segments]

import = True

path = ./data/reference_segments.pkl

[Corridors]

corridors =

altitude = 250

delta_z = 25

speed = 50

acceleration_length = 50

file_path_corridors = ./data/usepe-hannover-corridors.geojson

[Strategic_Deconfliction]

ratio = 3

delay = 60

In [3]:
# -------------- 1. CONFIGURATION FILE -----------------
"""
This section reads the configuration file.
Change the config_path to read the desired file
"""
# CONFIG
config_path = default_path2 + r"\settings_exercise_1_reference.cfg"
config = configparser.ConfigParser()
config.read( config_path )

['C:\\workspace3\\scenarios-exercise1\\scenario\\USEPE\\exercise_1\\settings_exercise_1_reference.cfg']

# Import the graph

In [4]:
# -------------- 2. CITY GRAPH -------------------------
"""
This section creates a city graph or loads the graph defined with the city section of the
configuration file.
"""
# City
os.chdir(default_path2)
if config['City'].getboolean( 'import' ):
    filepath = config['City']['imported_graph_path']
    G = read_my_graphml( filepath )
    G = MultiDiGrpah3D( G )
    # fig, ax = ox.plot_graph( G )
else:
    print('Check the path to the graph file')
    G = cityGraph( config )

Reading the graph...


# Import the segments

In [5]:
# -------------- 3. SEGMENTS ----------------------------
"""
This section creates a airspace segmentation or loads the segmentation defined with the segment
section of the configuration file.
Comment it to neglect the segmentation
"""
os.chdir(default_path2)
if config['Segments'].getboolean( 'import' ):
    path = config['Segments']['path']
    with open( path, 'rb' ) as f:
        segments = pickle.load( f )
else:
    segments = None

# Flight plan creation function

# Determine the path to the csv and create the flight plans

In [6]:
strategic_deconfliction = False

#path_csv = "./data/background_traffic_0-0964_544_7200.csv"  # background
path_csv = "./data/delivery_1800-1800-1800_None-1000-None_7200.csv"  # delivery

createFlightPlansFromCSV(default_path, path_csv, strategic_deconfliction, G, segments, config)

18
Calculating the route...
Calculating the route...
Creating delivery flight plan of D1...
18
Calculating the route...
Calculating the route...


KeyboardInterrupt: 